In [144]:
import sys
import pandas as pd
from sqlalchemy import create_engine
import sys
# import libraries
import pandas as pd
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import xgboost as xgb
import joblib
 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
pd.set_option('display.max_columns', None)

In [145]:
def load_data(portfolio_filepath, profile_filepath,transcript_filepath):
    """
    Loads the data to be processed

    INPUT:
    portfolio_filepath: file path of the porfolio json file 
    profile_filepath: file path of the profile json file 
    transcript_filepath: file path of the transcript json file

    OUTPUT:
    df: a dataframe combining the 3 json files
    """
    # load portfolio dataset
    portfolio = pd.read_json(portfolio_filepath, orient='records', lines=True)

    # load profile dataset
    profile = pd.read_json(profile_filepath, orient='records', lines=True)

    # load transcript dataset
    transcript = pd.read_json(transcript_filepath, orient='records', lines=True)


# ### 2. Merge datasets.
# - Merge the messages and categories datasets using the common id
# - Assign this combined dataset to `df`, which will be cleaned in the following steps


    return portfolio,profile,transcript

In [146]:
def clean_data(portfolio,profile,transcript):
    """
    Cleans the df spliting columns, build a 0/1 columns for each category, etc
    """
    transcript_norm = pd.json_normalize(transcript.value)
    transcript_norm.offer_id = transcript_norm[['offer id','offer_id']].bfill(axis=1).iloc[:, 0]
    transcript_norm = transcript_norm.drop(columns = ['offer id'])
    transcript_norm = pd.concat([transcript.drop(columns=['value']),transcript_norm],axis=1)

    # create a dataframe of the 36 individual category columns
    profile.became_member_on = pd.to_datetime(profile.became_member_on,format='%Y%m%d')

    profile['days_as_member'] = (profile.became_member_on.max()-profile.became_member_on).dt.days

    def age_informed(age):
        if pd.isna(age) or age==118:
            return False
        else:
            return True

    profile['income_informed'] = -profile.income.isna()
    profile['income_informed'] =profile['income_informed'].astype(int)
    profile.income = profile.income.fillna(profile.income.mean())
    profile['age_informed'] = profile.age.apply(lambda x: age_informed(x)).astype(int)
    profile['gender'] = profile['gender'].fillna('na')



    # select the first row of the categories dataframe
    df = transcript_norm.drop(columns='reward').merge(portfolio, left_on='offer_id',right_on='id',how='left').drop(columns='id')

    df = df.merge(profile.rename(columns={'id':'person'}), on='person',how='left')
    df = df.sort_values(by='time')

    df['offer_rank'] = df.sort_values(by=['person','offer_id','time','event']).groupby(['person','offer_id','event'])['time'].rank('dense')

    df_offer_amount = df[df.event=='offer completed'][['person','time']].merge(df[df.event=='transaction'][['person','time','amount']],on=['person','time'],how='left')

    df_offer_amount = df_offer_amount.rename(columns = {'amount':'offer_amount'})

    df = df.merge(df_offer_amount,on=['person','time'],how='left')

    #df['offer_amount'] = df.apply(lambda x: x.offer_amount if x.event=='offer completed' else None,axis =1)

    df['offer_amount'] = df['offer_amount'].fillna(df.groupby(['person','offer_id','offer_rank'])['offer_amount'].transform('mean'))

    df['days_since_offer'] = df.apply(lambda x: x.time if x.event=='offer received' else None,axis =1)
    df['days_since_offer'] = df.groupby('person')['days_since_offer'].transform(lambda x: x.ffill())
    df['days_since_offer'] = df.time - df.days_since_offer

    df['offer_success'] = df.apply(lambda x: 1 if x.offer_amount>0 and x.event=='offer received' else None,axis=1)
    df['offer_completed_hist'] = df.apply(lambda x: x.offer_amount if x.offer_amount>0 and x.event=='offer completed' else None,axis=1)

    dummies =['gender','offer_id']

    for var in  dummies:
        # for each cat add dummy var, drop original column
        df = pd.concat([df, pd.get_dummies(df[var], prefix=var, prefix_sep='_')], axis=1)

    list_offers_ids = [x for x in df.columns if x.startswith("offer_id_")]

    df[list_offers_ids] = df[list_offers_ids].multiply(df["offer_completed_hist"]*(df['event']=='offer completed'), axis="index")

    df[list_offers_ids] = df.fillna(0).groupby('person')[list_offers_ids].cumsum()

    overall_cum = df.fillna(0).groupby('person')[['amount','offer_success']].cumsum()
    df[['amount_cum','offer_success_cum']] = overall_cum

    # use this row to extract a list of new column names for categories.
    df_select_offer = df.groupby('person').time.max().reset_index().merge(df,on=['person','time'])

    df_select_offer.drop_duplicates(subset='person',keep='last',inplace=True)

    df['selected_offer'] = df.apply(lambda x: x.offer_id if x.offer_success==1 else 'offer_fail',axis =1)

    last_info = df.groupby('person').time.max().reset_index()
    last_info['last_info'] =1

    df = df.merge(last_info,on=['person','time'],how='left')
    df.last_info = df.last_info.fillna(0)

    df.drop(columns= ['channels'],inplace=True)
    # rename the columns of `categories`
    return df

In [147]:
def save_data(df, database_filename,table_name):
    """
    Saves the DataFrame into a database

    INPUT:
    df: dataframe
    database_filename: name of the database to be saved
    """
    engine = create_engine('sqlite:///{}'.format(database_filename))
    df.to_sql(table_name, engine, index=False, if_exists='replace')

In [165]:
def load_data2(database_filepath):
    """
    INPUT:
    database_filepath: the file path to the database

    OUTPUT:
    X: features of the message df
    y: classifications of the messages
    col_list: list of classification columns
    """
    engine = create_engine('sqlite:///{}'.format(database_filepath))

    cols_drop =['person','selected_offer',
                            'last_info','event','became_member_on'
                            ,'offer_id', 'offer_type','gender',
               'amount','reward','difficulty','duration']
                            
    df = pd.read_sql("SELECT * FROM User", engine)
    df = df[df.event=='offer received'].copy()
    X = df.drop(columns =cols_drop).fillna(0)
    y = df['selected_offer']
    return X,y

In [155]:
def build_model():

    """
    builds a pipeline to be used for modeling a df
    """
    pipeline = Pipeline([ ('classifier', xgb.XGBClassifier(  eval_metric='mlogloss'))])


    param = {
        'classifier__max_depth':[2,4,6,8] 
        }
    cv = GridSearchCV(estimator =pipeline, param_grid =param )



    return cv

In [156]:
def evaluate_model(model, X_test, Y_test):
    """
    INPUT:
    model: model to be applied on the df
    X_test: part of the df to be used as featues
    Y_text: response part of the df
    category_names: names pf the categories

    OUTPUT:
    print the repport comparing the test response data (Y) with the model applied to the test data (X)
    """

    y_pred = model.best_estimator_.predict(X_test)

    print(classification_report(Y_test, y_pred))

In [157]:
def save_model(model, model_filepath):
    """
    Saves the model into a pickle file

    INPUT:
    model: model to be saved
    model_filepath: file path to save the model on
    """
    joblib.dump(model.best_estimator_, model_filepath)

In [148]:
portfolio_filepath = 'data/portfolio.json'
profile_filepath = 'data/profile.json'
transcript_filepath =  'data/transcript.json'
database_filepath  = 'data/User.db'
model_filepath = 'models/classifier.pkl'

In [149]:
print('Loading data...\n    PORTFOLIO: {}\n    PROFILE: {}\n    TRANSCRIPT: {}'
      .format(portfolio_filepath, profile_filepath,transcript_filepath))
portfolio,profile,transcript = load_data(portfolio_filepath, profile_filepath,transcript_filepath)

Loading data...
    PORTFOLIO: data/portfolio.json
    PROFILE: data/profile.json
    TRANSCRIPT: data/transcript.json


In [150]:
print('Cleaning data...')
df = clean_data(portfolio,profile,transcript)

Cleaning data...


In [152]:
print('Saving data...\n    DATABASE: {}'.format(database_filepath))
save_data(df, database_filepath,'User')

Saving data...
    DATABASE: data/User.db


In [166]:
print('Loading data...\n    DATABASE: {}'.format(database_filepath))
X, Y = load_data2(database_filepath)

Loading data...
    DATABASE: data/User.db


In [167]:
X.head()

,time,age,income,days_as_member,income_informed,age_informed,offer_rank,offer_amount,days_since_offer,offer_success,offer_completed_hist,gender_F,gender_M,gender_O,gender_na,offer_id_0b1e1539f2cc45b7b9fa7c272da2e1d7,offer_id_2298d6c36e964ae4a3e7e9706d1fb8c2,offer_id_2906b810c7d4411798c6938adc9daaa5,offer_id_3f207df678b143eea3cee63160fa8bed,offer_id_4d5c57ea9a6940dd891ad53e9dbe8da0,offer_id_5a8bc65990b245e5a138643cd4eb9837,offer_id_9b98b8c7a33c4b65b9aebfe6a799e6d9,offer_id_ae264e3637204a6fb9bb56bc8210ddfd,offer_id_f19421c1d4aa40978ebb69ca19b0e20d,offer_id_fafdcd668e3743c1bb461111dcafc2a4,amount_cum,offer_success_cum
0,0,75,100000.000000,443,1,1,1.0,19.89,0.0,1.0,0.0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,118,65404.991568,188,0,0,1.0,0.00,0.0,0.0,0.0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,57,59000.000000,254,1,1,1.0,40.06,0.0,1.0,0.0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0,73,39000.000000,109,1,1,1.0,0.00,0.0,0.0,0.0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,64,70000.000000,290,1,1,1.0,0.00,0.0,0.0,0.0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [160]:
Y

0         9b98b8c7a33c4b65b9aebfe6a799e6d9
1                               offer_fail
2         fafdcd668e3743c1bb461111dcafc2a4
3                               offer_fail
4                               offer_fail
                        ...               
268875                          offer_fail
268876                          offer_fail
268877                          offer_fail
268878    2298d6c36e964ae4a3e7e9706d1fb8c2
268879    9b98b8c7a33c4b65b9aebfe6a799e6d9
Name: selected_offer, Length: 76668, dtype: object

In [168]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [169]:
print('Building model...')
model = build_model()

Building model...


In [170]:
print('Training model...')
model.fit(X_train, Y_train)

Training model...


C:\Users\bruno\anaconda3\envs\py37\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


GridSearchCV(estimator=Pipeline(steps=[('classifier',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      eval_metric='mlogloss',
                                                      gamma=None, gpu_id=None,
                                                      importance_type='gain',
                                                      interaction_constraints=None,
                                                      learning_rate=None,
                                                      max_delta_step=None,
                                                      max_depth=None,
                                     

In [171]:
print('Evaluating model...')
evaluate_model(model, X_test, Y_test)

Evaluating model...
                                  precision    recall  f1-score   support

0b1e1539f2cc45b7b9fa7c272da2e1d7       0.19      0.31      0.23       695
2298d6c36e964ae4a3e7e9706d1fb8c2       0.31      0.28      0.30      1048
2906b810c7d4411798c6938adc9daaa5       0.28      0.12      0.17       827
3f207df678b143eea3cee63160fa8bed       0.00      0.00      0.00        24
4d5c57ea9a6940dd891ad53e9dbe8da0       0.25      0.19      0.21       681
5a8bc65990b245e5a138643cd4eb9837       0.50      0.05      0.08        22
9b98b8c7a33c4b65b9aebfe6a799e6d9       0.28      0.25      0.26       857
ae264e3637204a6fb9bb56bc8210ddfd       0.20      0.24      0.22       747
f19421c1d4aa40978ebb69ca19b0e20d       0.31      0.27      0.29       847
fafdcd668e3743c1bb461111dcafc2a4       0.27      0.36      0.31      1076
                      offer_fail       1.00      1.00      1.00      8510

                        accuracy                           0.67     15334
                

In [143]:
print('Saving model...\n    MODEL: {}'.format(model_filepath))
save_model(model, model_filepath)

In [143]:
print('Trained model saved!')

ValueError: not enough values to unpack (expected 4, got 2)

In [ ]:
df = pd.read_sql("SELECT * FROM User", engine)

In [225]:
df_demo = df.drop_duplicates(subset=['person','gender','age']).copy()

In [211]:
genre_counts = df_demo.groupby('gender').count()['person']
genre_names = list(genre_counts.index)

In [244]:
df_demo['ages'] = pd.cut(x=df_demo['age'], bins=list(range(18,128,10)))
df_demo['ages'] = df_demo.apply(lambda x: x.ages if x.age<118 else 'not informed',axis=1)

ages = df_demo.groupby('ages')['person'].count()

ages_hist = ages
ages_names = [str(i) for i in ages_hist.index]

In [200]:
offers_sent = df[df.event=='offer received'].groupby('offer_id')[['person']].count().rename(columns={'person':'offers sent'})

offer_suc = df[df.offer_success	==1].groupby('offer_id')[['person']].count().rename(columns={'person':'offer success'})

df_success = offer_suc.merge(offers_sent,left_index=True,right_index=True)

df_success['sucess_rate'] = df_success['offer success']/df_success['offers sent']

In [246]:
df[[person]]

,person,event,time,amount,offer_id,reward,difficulty,duration,offer_type,gender,age,became_member_on,income,days_as_member,income_informed,age_informed,offer_rank,offer_amount,days_since_offer,offer_success,offer_completed_hist,gender_F,gender_M,gender_O,gender_na,offer_id_0b1e1539f2cc45b7b9fa7c272da2e1d7,offer_id_2298d6c36e964ae4a3e7e9706d1fb8c2,offer_id_2906b810c7d4411798c6938adc9daaa5,offer_id_3f207df678b143eea3cee63160fa8bed,offer_id_4d5c57ea9a6940dd891ad53e9dbe8da0,offer_id_5a8bc65990b245e5a138643cd4eb9837,offer_id_9b98b8c7a33c4b65b9aebfe6a799e6d9,offer_id_ae264e3637204a6fb9bb56bc8210ddfd,offer_id_f19421c1d4aa40978ebb69ca19b0e20d,offer_id_fafdcd668e3743c1bb461111dcafc2a4,amount_cum,offer_success_cum,selected_offer,last_info
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,5.0,7.0,bogo,F,75,2017-05-09 00:00:00.000000,100000.000000,443,1,1,1.0,19.89,0.0,1.0,NaN,1,0,0,0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,1.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0
1,da6c67094d3c4612b1146e707d8ba1ac,offer received,0,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,10.0,10.0,5.0,bogo,na,118,2018-01-19 00:00:00.000000,65404.991568,188,0,0,1.0,NaN,0.0,NaN,NaN,0,0,0,1,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,offer_fail,0.0
2,8c3be843043c4ed9893ad9aef2a3c71a,offer received,0,NaN,fafdcd668e3743c1bb461111dcafc2a4,2.0,10.0,10.0,discount,M,57,2017-11-14 00:00:00.000000,59000.000000,254,1,1,1.0,40.06,0.0,1.0,NaN,0,1,0,0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,1.0,fafdcd668e3743c1bb461111dcafc2a4,0.0
3,b91c99612ba54860b8702522d5431904,offer received,0,NaN,3f207df678b143eea3cee63160fa8bed,0.0,0.0,4.0,informational,M,73,2018-04-08 00:00:00.000000,39000.000000,109,1,1,1.0,NaN,0.0,NaN,NaN,0,1,0,0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,offer_fail,0.0
4,7fc0bf97fd3e4153b7a4022d7266e9f7,offer received,0,NaN,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0,7.0,7.0,discount,F,64,2017-10-09 00:00:00.000000,70000.000000,290,1,1,1.0,NaN,0.0,NaN,NaN,1,0,0,0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,offer_fail,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316327,fac959096aa4492db3b1114bc6f2cd27,transaction,714,2.70,None,NaN,NaN,NaN,None,M,58,2014-03-05 00:00:00.000000,67000.000000,1604,1,1,120.0,NaN,138.0,NaN,NaN,0,1,0,0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,16.19,0.0,68.47,2.0,offer_fail,1.0
316328,dd2f15d3224349b3a54cbb4a91e89e0c,transaction,714,17.51,None,NaN,NaN,NaN,None,F,65,2016-05-17 00:00:00.000000,66000.000000,800,1,1,120.0,NaN,138.0,NaN,NaN,1,0,0,0,0.00,16.13,11.08,0.0,0.0,0.0,0.00,13.63,32.61,0.0,321.57,4.0,offer_fail,1.0
316329,df72761941ab427b9c8878508ad5814e,offer completed,714,NaN,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0,7.0,7.0,discount,F,18,2013-10-07 00:00:00.000000,53000.000000,1753,1,1,1.0,4.19,138.0,NaN,4.19,1,0,0,0,1.73,4.19,0.00,0.0,0.0,0.0,14.16,0.00,14.16,0.0,46.60,4.0,offer_fail,1.0
316330,93372c50aa324bc799b9d5b74070ea21,transaction,714,13.21,None,NaN,NaN,NaN,None,O,43,2015-08-25 00:00:00.000000,78000.000000,1066,1,1,120.0,NaN,138.0,NaN,NaN,0,0,1,0,0.00,7.23,0.00,0.0,0.0,0.0,23.60,22.11,0.00,0.0,237.83,3.0,offer_fail,1.0


In [138]:
X,Y = load_data('data/User.db')

In [139]:
X.head()

,time,amount,offer_id,reward,difficulty,duration,offer_type,gender,age,income,days_as_member,income_informed,age_informed,offer_rank,offer_amount,days_since_offer,offer_success,offer_completed_hist,gender_F,gender_M,gender_O,gender_na,offer_id_0b1e1539f2cc45b7b9fa7c272da2e1d7,offer_id_2298d6c36e964ae4a3e7e9706d1fb8c2,offer_id_2906b810c7d4411798c6938adc9daaa5,offer_id_3f207df678b143eea3cee63160fa8bed,offer_id_4d5c57ea9a6940dd891ad53e9dbe8da0,offer_id_5a8bc65990b245e5a138643cd4eb9837,offer_id_9b98b8c7a33c4b65b9aebfe6a799e6d9,offer_id_ae264e3637204a6fb9bb56bc8210ddfd,offer_id_f19421c1d4aa40978ebb69ca19b0e20d,offer_id_fafdcd668e3743c1bb461111dcafc2a4,amount_cum,offer_success_cum
0,0,0.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,5.0,7.0,bogo,F,75,100000.000000,443,1,1,1.0,19.89,0.0,1.0,0.0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,0.0,4d5c57ea9a6940dd891ad53e9dbe8da0,10.0,10.0,5.0,bogo,na,118,65404.991568,188,0,0,1.0,0.00,0.0,0.0,0.0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.0,fafdcd668e3743c1bb461111dcafc2a4,2.0,10.0,10.0,discount,M,57,59000.000000,254,1,1,1.0,40.06,0.0,1.0,0.0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0,0.0,3f207df678b143eea3cee63160fa8bed,0.0,0.0,4.0,informational,M,73,39000.000000,109,1,1,1.0,0.00,0.0,0.0,0.0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.0,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0,7.0,7.0,discount,F,64,70000.000000,290,1,1,1.0,0.00,0.0,0.0,0.0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [17]:
model = build_model()

In [ ]:
pipeline = Pipeline([ ('classifier',MultiOutputClassifier(xgb.XGBClassifier()))])


param = {
'classifier__max_depth':[2,4,8,10] 
}

cv = GridSearchCV(estimator =pipeline, param_grid =param )


cv.fit(X_train, Y_train)

In [90]:
pipeline = Pipeline([ ('classifier', xgb.XGBClassifier(  eval_metric='mlogloss'))])


param = {
    'classifier__max_depth':[2,4,8,10] 
    }
cv = GridSearchCV(estimator =pipeline, param_grid =param )

cv.fit(X_train, Y_train)

C:\Users\bruno\anaconda3\envs\py37\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\bruno\anaconda3\envs\py37\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GridSearchCV(estimator=Pipeline(steps=[('classifier',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      eval_metric='mlogloss',
                                                      gamma=None, gpu_id=None,
                                                      importance_type='gain',
                                                      interaction_constraints=None,
                                                      learning_rate=None,
                                                      max_delta_step=None,
                                                      max_depth=None,
                                     

In [88]:
cv.best_estimator_

Pipeline(steps=[('classifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, eval_metric='mlogloss',
                               gamma=0, gpu_id=-1, importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=4, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=12, num_parallel_tree=1,
                               objective='multi:softprob', random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
                               subsample=1, tree_method='exact',
                               validate_parameters=1, verbosity=None))])

In [85]:
pipeline = Pipeline([ ('classifier', xgb.XGBClassifier(  eval_metric='mlogloss'))])

score = 'accuracy'

param = {
    'classifier__max_depth':[2,4,8,10] 
    }
cv = GridSearchCV(estimator =pipeline, param_grid =param , scoring= score)

cv.fit(X_train, Y_train)
cv.best_estimator_.predict(X_test)

C:\Users\bruno\anaconda3\envs\py37\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\bruno\anaconda3\envs\py37\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


array(['offer_fail', '2298d6c36e964ae4a3e7e9706d1fb8c2', 'offer_fail',
       ..., 'offer_fail', 'fafdcd668e3743c1bb461111dcafc2a4',
       'f19421c1d4aa40978ebb69ca19b0e20d'], dtype=object)

In [86]:
cv.best_estimator_

Pipeline(steps=[('classifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, eval_metric='mlogloss',
                               gamma=0, gpu_id=-1, importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=4, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=12, num_parallel_tree=1,
                               objective='multi:softprob', random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
                               subsample=1, tree_method='exact',
                               validate_parameters=1, verbosity=None))])

In [ ]:
        
        print('Building model...')
        model = build_model()
        
        print('Training model...')
        model.fit(X_train, Y_train)

In [ ]:
load_data('data/portfolio.json','data/profile.json','data/transcript.json')

In [91]:
X

,time,age,days_as_member,income,income_informed,age_informed,days_since_offer,offer_completed_hist,gender_F,gender_M,...,offer_id_2906b810c7d4411798c6938adc9daaa5,offer_id_3f207df678b143eea3cee63160fa8bed,offer_id_4d5c57ea9a6940dd891ad53e9dbe8da0,offer_id_5a8bc65990b245e5a138643cd4eb9837,offer_id_9b98b8c7a33c4b65b9aebfe6a799e6d9,offer_id_ae264e3637204a6fb9bb56bc8210ddfd,offer_id_f19421c1d4aa40978ebb69ca19b0e20d,offer_id_fafdcd668e3743c1bb461111dcafc2a4,amount_cum,offer_success_cum
0,0,75,443,100000.000000,1,1,0.0,0.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,1.0
1,0,118,188,65404.991568,0,0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0
2,0,57,254,59000.000000,1,1,0.0,0.0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,1.0
3,0,73,109,39000.000000,1,1,0.0,0.0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0
4,0,64,290,70000.000000,1,1,0.0,0.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76663,576,54,100,72000.000000,1,1,0.0,0.0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,7.87,0.00,39.68,1.0
76664,576,36,1646,40000.000000,1,1,0.0,0.0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,5.18,44.49,3.0
76665,576,26,34,62000.000000,1,1,0.0,0.0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,13.21,0.0
76666,576,44,840,64000.000000,1,1,0.0,0.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,18.13,134.41,3.0


In [260]:
engine = create_engine('sqlite:///{}'.format('data/User.db'))
df = pd.read_sql("SELECT * FROM User", engine)
df.shape

(316332, 39)

In [106]:
df = df.groupby('person').time.max().reset_index().merge(df,on=['person','time'])

In [109]:
df.head()

,person,time,age,days_as_member,income,income_informed,age_informed,days_since_offer,offer_completed_hist,gender_F,...,offer_id_3f207df678b143eea3cee63160fa8bed,offer_id_4d5c57ea9a6940dd891ad53e9dbe8da0,offer_id_5a8bc65990b245e5a138643cd4eb9837,offer_id_9b98b8c7a33c4b65b9aebfe6a799e6d9,offer_id_ae264e3637204a6fb9bb56bc8210ddfd,offer_id_f19421c1d4aa40978ebb69ca19b0e20d,offer_id_fafdcd668e3743c1bb461111dcafc2a4,amount_cum,offer_success_cum,selected_offer
0,0009655768c64bdeb2e877511632db8f,576,33,461,72000.000000,1,1,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,8.57,14.11,68.67,3.0,2906b810c7d4411798c6938adc9daaa5
1,00116118485d4dfda04fdbaba9a87b5c,576,118,92,65404.991568,0,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,4.09,0.0,offer_fail
2,0011e0d4e6b944f998e987f904e8c1e5,504,40,198,57000.000000,1,1,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,25.42,3.0,9b98b8c7a33c4b65b9aebfe6a799e6d9
3,0020c2b971eb4e9188eac86d93036a77,504,59,874,90000.000000,1,1,0.0,0.0,1,...,0.0,0.0,0.0,0.0,0.0,0.00,17.63,98.33,3.0,offer_fail
4,0020ccbbb6d84e358d3414a3ff76cffd,504,24,622,60000.000000,1,1,0.0,0.0,1,...,0.0,0.0,0.0,0.0,0.0,14.53,0.00,136.26,3.0,9b98b8c7a33c4b65b9aebfe6a799e6d9


In [125]:
columns_save = [ 'person','time', 'age',
       'days_as_member', 'income', 'income_informed', 'age_informed',
          'days_since_offer', 
       'offer_completed_hist', 'gender_F', 'gender_M', 'gender_O', 'gender_na',
       'offer_id_0b1e1539f2cc45b7b9fa7c272da2e1d7',
       'offer_id_2298d6c36e964ae4a3e7e9706d1fb8c2',
       'offer_id_2906b810c7d4411798c6938adc9daaa5',
       'offer_id_3f207df678b143eea3cee63160fa8bed',
       'offer_id_4d5c57ea9a6940dd891ad53e9dbe8da0',
       'offer_id_5a8bc65990b245e5a138643cd4eb9837',
       'offer_id_9b98b8c7a33c4b65b9aebfe6a799e6d9',
       'offer_id_ae264e3637204a6fb9bb56bc8210ddfd',
       'offer_id_f19421c1d4aa40978ebb69ca19b0e20d',
       'offer_id_fafdcd668e3743c1bb461111dcafc2a4', 'amount_cum',
       'offer_success_cum','selected_offer']

In [127]:
[x for x in columns_save if x!='selected_offer']

['person',
 'time',
 'age',
 'days_as_member',
 'income',
 'income_informed',
 'age_informed',
 'days_since_offer',
 'offer_completed_hist',
 'gender_F',
 'gender_M',
 'gender_O',
 'gender_na',
 'offer_id_0b1e1539f2cc45b7b9fa7c272da2e1d7',
 'offer_id_2298d6c36e964ae4a3e7e9706d1fb8c2',
 'offer_id_2906b810c7d4411798c6938adc9daaa5',
 'offer_id_3f207df678b143eea3cee63160fa8bed',
 'offer_id_4d5c57ea9a6940dd891ad53e9dbe8da0',
 'offer_id_5a8bc65990b245e5a138643cd4eb9837',
 'offer_id_9b98b8c7a33c4b65b9aebfe6a799e6d9',
 'offer_id_ae264e3637204a6fb9bb56bc8210ddfd',
 'offer_id_f19421c1d4aa40978ebb69ca19b0e20d',
 'offer_id_fafdcd668e3743c1bb461111dcafc2a4',
 'amount_cum',
 'offer_success_cum']

In [123]:
columns_save+'selected_offer'

TypeError: can only concatenate list (not "str") to list

In [124]:
engine = create_engine('sqlite:///{}'.format('data/User.db'))
df = pd.read_sql("SELECT * FROM User", engine)
df

,person,time,age,days_as_member,income,income_informed,age_informed,days_since_offer,offer_completed_hist,gender_F,...,offer_id_3f207df678b143eea3cee63160fa8bed,offer_id_4d5c57ea9a6940dd891ad53e9dbe8da0,offer_id_5a8bc65990b245e5a138643cd4eb9837,offer_id_9b98b8c7a33c4b65b9aebfe6a799e6d9,offer_id_ae264e3637204a6fb9bb56bc8210ddfd,offer_id_f19421c1d4aa40978ebb69ca19b0e20d,offer_id_fafdcd668e3743c1bb461111dcafc2a4,amount_cum,offer_success_cum,selected_offer
0,78afa995795e4d85b5d9ceeca43f5fef,0,75,443,100000.000000,1,1,0.0,0.0,1,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,1.0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,da6c67094d3c4612b1146e707d8ba1ac,0,118,188,65404.991568,0,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,offer_fail
2,8c3be843043c4ed9893ad9aef2a3c71a,0,57,254,59000.000000,1,1,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,1.0,fafdcd668e3743c1bb461111dcafc2a4
3,b91c99612ba54860b8702522d5431904,0,73,109,39000.000000,1,1,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,offer_fail
4,7fc0bf97fd3e4153b7a4022d7266e9f7,0,64,290,70000.000000,1,1,0.0,0.0,1,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,offer_fail
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76663,f4996c5638134185b27f27e8bb0adaf1,576,54,100,72000.000000,1,1,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,7.87,0.00,39.68,1.0,offer_fail
76664,f70e4e136db845fc8d9999a5c5941478,576,36,1646,40000.000000,1,1,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.00,5.18,44.49,3.0,offer_fail
76665,a1b42659653a462ba62fed071e1f4284,576,26,34,62000.000000,1,1,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,13.21,0.0,offer_fail
76666,2a1cb179fd8f4dc8b44949eeced7d7b1,576,44,840,64000.000000,1,1,0.0,0.0,1,...,0.0,0.0,0.0,0.0,0.0,0.00,18.13,134.41,3.0,2298d6c36e964ae4a3e7e9706d1fb8c2


In [248]:
import os
os.getcwd()

'C:\\Users\\bruno\\OneDrive\\Documentos\\GitHub\\StarbucksCapstone\\workspace'

In [263]:
df[df.last_info==1].shape

(21383, 39)

In [277]:
model = joblib.load("models/classifier.pkl")

cols_drop =['selected_offer',
                        'last_info','event','became_member_on'
                        ,'offer_id', 'offer_type','gender',
            'amount','reward','difficulty','duration']
                            
X = df[df.last_info==1].copy()
X = X.drop(columns =cols_drop).fillna(0)

classification_labels = model.predict(X.drop(columns=['person']))

X['selected offer'] = classification_labels

X[['person', 'selected offer']]

In [279]:
X[['person', 'selected offer']].to_csv('X.csv')

In [281]:
df.to_csv('df.csv')

In [287]:
X[['person', 'selected offer']].head(5).values.tolist()

[['6cf51740b6e94bd09e738b70f3d6c1c8', 'offer_fail'],
 ['27793a3c6dfe47cda8ee29432c6f6603', '5a8bc65990b245e5a138643cd4eb9837'],
 ['e959acbfc347460b8a55f94fc64e661b', 'offer_fail'],
 ['df3181a5f3cf459f91a82c8f30889b1a', 'offer_fail'],
 ['7ecfc592171f4844bdc05bdbb48d3847', 'offer_fail']]

In [291]:
[X[k].tolist() for k in ['person', 'selected offer']]

[['6cf51740b6e94bd09e738b70f3d6c1c8',
  '27793a3c6dfe47cda8ee29432c6f6603',
  'e959acbfc347460b8a55f94fc64e661b',
  'df3181a5f3cf459f91a82c8f30889b1a',
  '7ecfc592171f4844bdc05bdbb48d3847',
  '9894250fd5b34abda073bb1c71f19f0d',
  'dd1ba9ff27e64ccbbe28566ff10189d9',
  '7ecfc592171f4844bdc05bdbb48d3847',
  'df3181a5f3cf459f91a82c8f30889b1a',
  'fafcd6ee168140fbbb5da43be1d3daa7',
  '9fb2081cc2eb4cb9bfb7265dca9a7879',
  '50f26d1787444bb6a1e1b8be501f6877',
  'ffed75d3abc64b488982f50ed12878b5',
  'f6e7986202724b9cb7000be668d10359',
  '8778b4fe9ab94f04888cbac6b883b406',
  'bc0c484263b94b0896f20c5e4fdf3585',
  'db3d9ca5a8f54526bd5e263d914f3b29',
  'd1c50a0b12b84565a5182ee69f6c773e',
  '60ae06998f034851b0c571f8fb4e1165',
  '1bfe13d2453c4185a6486c6817e0d568',
  'f7178f6391fc4b698ea2e04bd7ffe512',
  'ad0884c461704fcdad54dc7d8150ae32',
  'a2dd3153f91042cbbed1bd60d99716ef',
  'b52302342d674c9784218a66bf550cca',
  '99129a5d44304888a2156e5c923cfed6',
  '6ac3a35e06d848e9b02ec09e74d1ff7b',
  '4d8213f88